In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fruit-and-vegetable-image-recognition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F952827%2F3173719%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T123316Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D35a97ceef848925a148a5c931bbb73c2a2cb17babad2287d87eb01b93088d8d8fb9e6ae66a50fdc25c5d56fb301d2601eaa868ed22b9f7e9691aa2ec9670aeb5a926eeb1f0df65b1dd555b11e6a345e90b9fe6c697f63fb150f91dc4ff82360c0f80bb3aaa5eac91edb1139d7738b8be4f65d261dea1afea47a6d361aa7858e362c422499c1556e65e5be0d0477aa2e2f4abb51cb31d5a4cc14fc1086732911f1bbb087c0a30989922b32af9daf6cee8d97af99c6e085a3a54dabae907a78b22280a6593d82d397edc3dbc9de682e3b323a691843c2dfa27bd76fb2f71de4d59dd0d743f6889e6bcbe070a28390f3111e019b2aaea2543aa733f5e6e59445c9b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2130757290 bytes downloaded
Downloaded and uncompressed: fruit-and-vegetable-image-recognition
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# DATA LOAD

In [ ]:
train_dir = '/kaggle/input/fruit-and-vegetable-image-recognition/train'
test_dir = '/kaggle/input/fruit-and-vegetable-image-recognition/test'
val_dir = '/kaggle/input/fruit-and-vegetable-image-recognition/validation'

In [ ]:
batch_size = 32
image_size = 160

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(image_size, image_size),
                                                                 batch_size=batch_size)

val_data = tf.keras.preprocessing.image_dataset_from_directory(val_dir,
                                                                 image_size=(image_size, image_size),
                                                                 batch_size=batch_size)

In [ ]:
class_name = train_data.class_names
print(class_name)

image samples visualization

In [ ]:
plt.figure(figsize=(10, 10))

for images, labels in train_data.take(1):
  for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_name[labels[i]])
    plt.axis('off')


check the shape of data

In [ ]:
for image_batch, label_batch in train_data:
  print(image_batch.shape)
  print(label_batch.shape)
  break

In [ ]:
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

make data between 0-1

In [ ]:
def format(image, label):
  image = tf.cast(image, tf.float32)
  image = image/255.0
  return image, label

In [ ]:
train = train_data.map(format)
validation = val_data.map(format)

In [ ]:
for format_image_batch, format_label_batch in train:
  print(format_image_batch.shape)
  print(format_label_batch.shape)
  break

In [ ]:
format_first_image = format_image_batch[0]
print(np.min(format_first_image), np.max(format_first_image))

# MODELING

use MobileNetV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape = (image_size, image_size, 3),
                                               include_top = False,
                                               weights = 'imagenet')

size check

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = keras.layers.Dense(36)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
model = tf.keras.Sequential([
        base_model,
        global_average_layer,
        prediction_layer
])

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train,
                    epochs=3,
                    validation_data=validation)

# MODEL EVALUATE

In [ ]:
def plot_graghs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel('Epochs')
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [ ]:
plot_graghs(history, 'accuracy')

In [ ]:
plot_graghs(history, 'loss')

In [ ]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                image_size=(image_size, image_size),
                                                                batch_size = batch_size,
                                                                shuffle=False)

# TEST

In [ ]:
test = test_data.map(format)

In [ ]:
result = model.evaluate(test)
print("Test loss, Test accuracy : ", result)